In [36]:
import pandas as pd

file_path = 'C:\\Users\\XUN\\Desktop\\project_myself\\MRegularSeasonDetailedResults.csv'
df = pd.read_csv(file_path)

# 筛选出第133天及以前的数据
df_filtered = df[df['DayNum'] <= 133]

# 显示筛选后的前几行数据
df_filtered.head()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [37]:
# Group the data by season
grouped_by_season = df_filtered.groupby('Season')

# This dictionary will store the grouped dataframes by season
grouped_dataframes = {season: group for season, group in grouped_by_season}

# Displaying the keys to confirm the grouping
list(grouped_dataframes.keys())


[2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024]

建立表格

In [38]:
# Create a new DataFrame for each year with unique team IDs
team_dataframes = {}

for season, group in grouped_dataframes.items():
    # Extract unique team IDs (both winners and losers)
    unique_teams = pd.concat([group['WTeamID'], group['LTeamID']]).unique()
    unique_teams.sort()  # Sorting the team IDs for readability
    # Create a new DataFrame with these team IDs
    team_df = pd.DataFrame(unique_teams, columns=['TeamID'])
    # Store this DataFrame in the dictionary
    team_dataframes[season] = team_df

# Display an example DataFrame for one season to check the structure
team_dataframes[2006].head()


,TeamID
0,1102
1,1103
2,1104
3,1105
4,1106


加入获胜率

In [39]:
# Iterate over each season's DataFrame to calculate win rates for each team
for season, group in grouped_dataframes.items():
    # Calculate win counts and loss counts for each team
    win_counts = group['WTeamID'].value_counts()
    loss_counts = group['LTeamID'].value_counts()
    
    # Initialize a list to store win rates
    win_rates = []
    
    # Calculate win rate for each team in the season
    for team_id in team_dataframes[season]['TeamID']:
        wins = win_counts.get(team_id, 0)
        losses = loss_counts.get(team_id, 0)
        total_games = wins + losses
        win_rate = wins / total_games if total_games > 0 else 0
        win_rates.append(win_rate)
    
    # Add the win rates as a new column to the DataFrame
    team_dataframes[season]['WinRate'] = win_rates

# Display an example DataFrame for one season to check the new column
team_dataframes[2004].head()


,TeamID,WinRate
0,1102,0.785714
1,1103,0.464286
2,1104,0.586207
3,1105,0.448276
4,1106,0.533333


计算后面13个指标（都取平均值）

In [40]:
# Define the columns that represent the game statistics we're interested in
# Assuming these are the detailed stats columns for both winning and losing teams
stats_columns = [
    'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF'
]

# Iterate over each season to calculate average stats for each team
for season, group in grouped_dataframes.items():
    # Initialize a dictionary to store average stats for each team
    team_avg_stats = {team_id: {stat: 0 for stat in stats_columns} for team_id in team_dataframes[season]['TeamID']}
    
    # Calculate average stats for winning teams
    for stat in stats_columns:
        win_stats = group.groupby('WTeamID')[f'W{stat}'].mean()
        for team_id, avg in win_stats.items():
            if team_id in team_avg_stats:
                team_avg_stats[team_id][stat] = avg
                
    # Calculate average stats for losing teams, and combine with win stats for total average
    for stat in stats_columns:
        loss_stats = group.groupby('LTeamID')[f'L{stat}'].mean()
        for team_id, avg in loss_stats.items():
            if team_id in team_avg_stats:
                # Combine win and loss stats to get total average, weighted by number of games won/lost
                wins = group[group['WTeamID'] == team_id].shape[0]
                losses = group[group['LTeamID'] == team_id].shape[0]
                total_games = wins + losses
                if total_games > 0:  # Avoid division by zero
                    win_avg = team_avg_stats[team_id][stat] * (wins / total_games)
                    loss_avg = avg * (losses / total_games)
                    team_avg_stats[team_id][stat] = win_avg + loss_avg
    
    # Add the calculated averages to the DataFrame
    for stat in stats_columns:
        team_dataframes[season][stat] = team_dataframes[season]['TeamID'].apply(lambda id: team_avg_stats[id].get(stat, 0))





In [41]:
# Display an example DataFrame for one season to check the new columns
team_dataframes[2020].head()

,TeamID,WinRate,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,1101,0.592593,24.592593,56.074074,6.481481,20.592593,17.111111,23.037037,10.037037,21.814815,13.592593,14.740741,9.000000,2.666667,22.000000
1,1102,0.354839,24.838710,55.096774,8.580645,23.225806,13.967742,19.290323,7.419355,24.741935,14.483871,12.451613,4.967742,1.225806,16.870968
2,1103,0.758621,24.965517,57.137931,9.172414,25.310345,15.620690,20.344828,9.517241,27.620690,12.655172,12.586207,4.896552,2.862069,17.931034
3,1104,0.516129,27.548387,63.096774,10.774194,30.870968,16.129032,23.258065,11.709677,27.645161,14.225806,14.903226,6.322581,4.387097,20.225806
4,1105,0.266667,21.866667,57.566667,5.133333,18.266667,12.633333,18.833333,9.633333,24.833333,10.866667,13.133333,5.800000,2.100000,17.933333


加入排名的文件

In [42]:
# Load the Excel file to check its structure
excel_file_path = 'C:\\Users\\XUN\\Desktop\\project_myself\\MMasseyOrdinals.csv'
excel_df = pd.read_csv(excel_file_path)

# Display the first few rows to understand its structure
excel_df.head()


,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [43]:
# Filter out the first 10 days of ranking data for each season
filtered_rankings = excel_df[excel_df['RankingDayNum'] > 10]
# Initialize a dictionary to store the ranking statistics for each team and season
ranking_stats = {}

In [44]:
for season, group in filtered_rankings.groupby('Season'):
    team_stats = {}
    for team_id in team_dataframes[season]['TeamID']:
        team_ranks = group[group['TeamID'] == team_id]['OrdinalRank']
        if not team_ranks.empty:
            highest_rank = team_ranks.max()
            lowest_rank = team_ranks.min()
            median_rank = team_ranks.median()
            top_four_rank = team_ranks.quantile(0.25)  # Correctly representing the top 25%
            bottom_four_rank = team_ranks.quantile(0.75)  # Correctly representing the bottom 25%
            team_stats[team_id] = [highest_rank, top_four_rank, bottom_four_rank, median_rank, lowest_rank]

    # Fill the DataFrame with the calculated stats
    ranking_columns = ['HighestRank', 'TopFourRank', 'BottomFourRank', 'MedianRank', 'LowestRank']
    for index, row in team_dataframes[season].iterrows():
        team_id = row['TeamID']
        if team_id in team_stats:
            for i, stat in enumerate(ranking_columns):
                team_dataframes[season].at[index, stat] = team_stats[team_id][i]
        else:
            for stat in ranking_columns:
                team_dataframes[season].at[index, stat] = None  # Use None for teams without ranking data

# Display an example DataFrame for one season to check the corrections
team_dataframes[2003].head()


,TeamID,WinRate,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,HighestRank,TopFourRank,BottomFourRank,MedianRank,LowestRank
0,1102,0.428571,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,13.000000,11.428571,5.964286,1.785714,18.750000,219.0,130.00,160.00,149.0,52.0
1,1103,0.481481,27.148148,55.851852,5.444444,16.074074,19.037037,25.851852,9.777778,19.925926,15.222222,12.629630,7.259259,2.333333,19.851852,284.0,168.00,194.00,181.0,99.0
2,1104,0.607143,24.035714,57.178571,6.357143,19.857143,14.857143,20.928571,13.571429,23.928571,12.107143,13.285714,6.607143,3.785714,18.035714,106.0,18.00,34.75,30.0,1.0
3,1105,0.269231,24.384615,61.615385,7.576923,20.769231,15.423077,21.846154,13.500000,23.115385,14.538462,18.653846,9.307692,2.076923,20.230769,323.0,301.75,313.00,309.0,185.0
4,1106,0.464286,23.428571,55.285714,6.107143,17.642857,10.642857,16.464286,12.285714,23.857143,11.678571,17.035714,8.357143,3.142857,18.178571,307.0,229.00,261.25,245.0,166.0


In [45]:
team_dataframes[2005].head()

,TeamID,WinRate,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,HighestRank,TopFourRank,BottomFourRank,MedianRank,LowestRank
0,1102,0.586207,21.413793,47.724138,8.896552,24.413793,9.517241,13.068966,7.620690,15.413793,13.655172,10.068966,8.793103,1.724138,16.482759,198.0,69.75,103.0,88.0,9.0
1,1103,0.642857,24.785714,55.785714,8.142857,21.357143,13.892857,20.357143,10.285714,23.071429,16.107143,13.357143,6.750000,3.857143,18.821429,269.0,73.00,121.0,89.0,33.0
2,1104,0.766667,25.366667,52.666667,7.633333,19.566667,17.066667,23.266667,10.233333,26.466667,13.833333,12.666667,5.200000,4.433333,14.733333,86.0,15.00,23.0,18.0,7.0
3,1105,0.551724,23.655172,59.000000,6.103448,20.241379,15.655172,25.517241,12.931034,24.310345,13.137931,15.241379,10.620690,3.862069,19.344828,325.0,255.00,290.0,276.0,119.0
4,1106,0.500000,19.433333,51.000000,4.533333,15.100000,14.566667,22.433333,11.266667,22.400000,10.366667,15.766667,6.700000,2.000000,17.466667,322.0,241.75,287.0,265.0,91.0


In [46]:
# Extract the final rankings for day 133 for each team and season
final_rankings = filtered_rankings[filtered_rankings['RankingDayNum'] == 114]

# Initialize a dictionary to store the final ranking for each team by season
final_rank_by_season_team = {}

# Populate the dictionary
for season, season_df in final_rankings.groupby('Season'):
    final_rank_by_season_team[season] = dict(zip(season_df['TeamID'], season_df['OrdinalRank']))

# Add the final rankings to the team_dataframes
for season, season_df in team_dataframes.items():
    if season in final_rank_by_season_team:
        season_df['FinalRankDay133'] = season_df['TeamID'].apply(lambda x: final_rank_by_season_team[season].get(x, None))
    else:
        season_df['FinalRankDay133'] = None  # For seasons without final ranking data

# Display an example DataFrame for one season to check the new column
team_dataframes[2003].head()


,TeamID,WinRate,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,...,TO,Stl,Blk,PF,HighestRank,TopFourRank,BottomFourRank,MedianRank,LowestRank,FinalRankDay133
0,1102,0.428571,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,...,11.428571,5.964286,1.785714,18.750000,219.0,130.00,160.00,149.0,52.0,161
1,1103,0.481481,27.148148,55.851852,5.444444,16.074074,19.037037,25.851852,9.777778,19.925926,...,12.629630,7.259259,2.333333,19.851852,284.0,168.00,194.00,181.0,99.0,210
2,1104,0.607143,24.035714,57.178571,6.357143,19.857143,14.857143,20.928571,13.571429,23.928571,...,13.285714,6.607143,3.785714,18.035714,106.0,18.00,34.75,30.0,1.0,33
3,1105,0.269231,24.384615,61.615385,7.576923,20.769231,15.423077,21.846154,13.500000,23.115385,...,18.653846,9.307692,2.076923,20.230769,323.0,301.75,313.00,309.0,185.0,301
4,1106,0.464286,23.428571,55.285714,6.107143,17.642857,10.642857,16.464286,12.285714,23.857143,...,17.035714,8.357143,3.142857,18.178571,307.0,229.00,261.25,245.0,166.0,221


In [47]:
compact_results_file_path = 'C:\\Users\\XUN\\Desktop\\project_myself\\MNCAATourneyCompactResults.csv'
compact_results_df = pd.read_csv(compact_results_file_path)

# Filter the data to include only the seasons from 2003 onwards
compact_results_df_2003_onwards = compact_results_df[compact_results_df['Season'] >= 2003]

compact_results_df_2003_onwards.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
1136,2003,134,1421,92,1411,84,N,1
1137,2003,136,1112,80,1436,51,N,0
1138,2003,136,1113,84,1272,71,N,0
1139,2003,136,1141,79,1166,73,N,0
1140,2003,136,1143,76,1301,74,N,1


In [48]:
# Creating the newnewdata DataFrame based on the compact_results_df_2003_onwards data
# The DataFrame will include columns for team_1, team_2, and the match result where 1 indicates team_1 won, and 0 indicates team_2 won

# Initialize the new DataFrame
newnewdata = pd.DataFrame()

# Assign team_1 and team_2 based on winning and losing team IDs
newnewdata['team_1'] = compact_results_df_2003_onwards['WTeamID']
newnewdata['team_2'] = compact_results_df_2003_onwards['LTeamID']

# Determine the match result; since team_1 is always the winner in our setup, the result will always be 1
newnewdata['result'] = 1  # team_1 wins

newnewdata.head()


,team_1,team_2,result
1136,1421,1411,1
1137,1112,1436,1
1138,1113,1272,1
1139,1141,1166,1
1140,1143,1301,1


In [49]:
# Create a dictionary to hold a DataFrame for each season
newnewdata_by_season = {}

# Group the compact results data by season
for season, group in compact_results_df_2003_onwards.groupby('Season'):
    # Initialize the DataFrame for the season
    df_season = pd.DataFrame()
    
    # Assign team_1 and team_2
    df_season['team_1'] = group['WTeamID']
    df_season['team_2'] = group['LTeamID']
    
    # Since team_1 is always the winner in our setup, the result will always be 1 for this DataFrame
    df_season['result'] = 1  # team_1 wins
    
    # Store the DataFrame in the dictionary
    newnewdata_by_season[season] = df_season

In [50]:
print(newnewdata_by_season[2018])


      team_1  team_2  result
2117    1347    1254       1
2118    1382    1417       1
2119    1393    1113       1
2120    1411    1300       1
2121    1104    1439       1
...      ...     ...     ...
2179    1242    1181       1
2180    1437    1403       1
2181    1276    1260       1
2182    1437    1242       1
2183    1437    1276       1

[67 rows x 3 columns]


In [51]:
# Iterate over each season in newnewdata_by_season
for season, season_df in newnewdata_by_season.items():
    # Check if the season is in team_dataframes (it should be)
    if season in team_dataframes:
        team_info_df = team_dataframes[season]

        # For team_1, merge the team information from team_info_df based on team_1's ID
        season_df = season_df.merge(team_info_df, left_on='team_1', right_on='TeamID', how='left')
        # Rename the columns to indicate they are for team_1
        season_df.rename(columns=lambda x: x if x in ['team_1', 'team_2', 'result'] else f"team_1_{x}", inplace=True)
        
        # For team_2, repeat the process
        season_df = season_df.merge(team_info_df, left_on='team_2', right_on='TeamID', how='left')
        # Rename the columns to indicate they are for team_2
        season_df.rename(columns=lambda x: x if x in ['team_1', 'team_2', 'result', 'team_1_TeamID'] + [f"team_1_{col}" for col in team_info_df.columns if col != 'TeamID'] else f"team_2_{x}", inplace=True)
        
        # Drop the duplicate TeamID columns
        season_df.drop(['team_1_TeamID', 'team_2_TeamID'], axis=1, inplace=True)

        # Update the dictionary with the enhanced DataFrame
        newnewdata_by_season[season] = season_df





In [52]:
# Display an example enhanced DataFrame for one season to check the new columns
print(newnewdata_by_season[2018].head(5))

   team_1  team_2  result  team_1_WinRate  team_1_FGM  team_1_FGA  \
0    1347    1254       1        0.625000   23.281250   54.843750   
1    1382    1417       1        0.781250   26.375000   57.843750   
2    1393    1113       1        0.606061   22.757576   54.424242   
3    1411    1300       1        0.441176   25.823529   58.500000   
4    1104    1439       1        0.558824   25.294118   55.205882   

   team_1_FGM3  team_1_FGA3  team_1_FTM  team_1_FTA  ...  team_2_TO  \
0     7.656250    21.812500   12.718750   17.343750  ...  14.121212   
1     7.750000    19.468750   17.406250   23.093750  ...  11.843750   
2     5.848485    18.181818   16.181818   21.848485  ...  10.709677   
3     7.823529    21.529412   18.176471   25.294118  ...  14.161290   
4     6.382353    19.676471   15.382353   22.882353  ...  11.937500   

   team_2_Stl  team_2_Blk  team_2_PF  team_2_HighestRank  team_2_TopFourRank  \
0    5.575758    3.151515  19.636364               335.0               263.0  

In [53]:
print(newnewdata_by_season[2019])

    team_1  team_2  result  team_1_WinRate  team_1_FGM  team_1_FGA  \
0     1125    1396       1        0.833333   31.666667   63.733333   
1     1192    1341       1        0.580645   26.193548   55.225806   
2     1113    1385       1        0.687500   26.593750   59.468750   
3     1295    1300       1        0.516129   24.580645   54.709677   
4     1120    1308       1        0.735294   27.029412   60.441176   
..     ...     ...     ...             ...         ...         ...   
62    1120    1246       1        0.735294   27.029412   60.441176   
63    1277    1181       1        0.823529   27.764706   57.176471   
64    1403    1277       1        0.812500   26.156250   55.468750   
65    1438    1120       1        0.906250   25.625000   53.593750   
66    1438    1403       1        0.906250   25.625000   53.593750   

    team_1_FGM3  team_1_FGA3  team_1_FTM  team_1_FTA  ...  team_2_TO  \
0     10.366667    27.966667   13.166667   17.900000  ...  11.156250   
1      7.806452

In [54]:
for season, df in newnewdata_by_season.items():
    # Reorder columns to move 'result' to the end
    columns = [col for col in df.columns if col != 'result'] + ['result']
    newnewdata_by_season[season] = df[columns]

In [55]:
print(newnewdata_by_season[2018].columns)

Index(['team_1', 'team_2', 'team_1_WinRate', 'team_1_FGM', 'team_1_FGA',
       'team_1_FGM3', 'team_1_FGA3', 'team_1_FTM', 'team_1_FTA', 'team_1_OR',
       'team_1_DR', 'team_1_Ast', 'team_1_TO', 'team_1_Stl', 'team_1_Blk',
       'team_1_PF', 'team_1_HighestRank', 'team_1_TopFourRank',
       'team_1_BottomFourRank', 'team_1_MedianRank', 'team_1_LowestRank',
       'team_1_FinalRankDay133', 'team_2_WinRate', 'team_2_FGM', 'team_2_FGA',
       'team_2_FGM3', 'team_2_FGA3', 'team_2_FTM', 'team_2_FTA', 'team_2_OR',
       'team_2_DR', 'team_2_Ast', 'team_2_TO', 'team_2_Stl', 'team_2_Blk',
       'team_2_PF', 'team_2_HighestRank', 'team_2_TopFourRank',
       'team_2_BottomFourRank', 'team_2_MedianRank', 'team_2_LowestRank',
       'team_2_FinalRankDay133', 'result'],
      dtype='object')


In [56]:
import numpy as np

# Iterate over each season's DataFrame to randomly swap teams for 50% of the matches
for season, df in newnewdata_by_season.items():
    # Generate a random mask for selecting rows to swap
    swap_mask = np.random.rand(len(df)) < 0.5
    
    # Columns related to team_1 and team_2 that need to be swapped
    team_1_cols = [col for col in df.columns if 'team_1' in col]
    team_2_cols = [col for col in df.columns if 'team_2' in col]
    
    # Swap the team_1 and team_2 columns for rows identified by the mask
    df.loc[swap_mask, team_1_cols + team_2_cols] = df.loc[swap_mask, team_2_cols + team_1_cols].values
    
    # Update the result column for swapped rows: 1 becomes 0 since team_2 wins
    df.loc[swap_mask, 'result'] = 0

In [57]:
# Display an example DataFrame to verify the changes
print(newnewdata_by_season)

{2003:     team_1  team_2  team_1_WinRate  team_1_FGM  team_1_FGA  team_1_FGM3  \
0   1421.0  1411.0        0.448276   24.379310   56.793103     6.482759   
1   1436.0  1112.0        0.655172   24.827586   55.862069     5.275862   
2   1272.0  1113.0        0.793103   26.275862   60.000000     7.000000   
3   1166.0  1141.0        0.878788   28.696970   57.454545     7.969697   
4   1301.0  1143.0        0.600000   24.333333   53.333333     7.966667   
..     ...     ...             ...         ...         ...          ...   
59  1393.0  1328.0        0.827586   29.241379   62.206897     5.241379   
60  1277.0  1400.0        0.612903   23.032258   51.096774     4.903226   
61  1266.0  1242.0        0.821429   27.214286   56.250000     5.785714   
62  1393.0  1400.0        0.827586   29.241379   62.206897     5.241379   
63  1393.0  1242.0        0.827586   29.241379   62.206897     5.241379   

    team_1_FGA3  team_1_FTM  team_1_FTA  team_1_OR  ...  team_2_Stl  \
0     18.000000   15.

In [58]:
seeds_df = pd.read_csv('C:\\Users\\XUN\\Desktop\\project_myself\\2024_tourney_seeds.csv')


选出2024年的数据

In [59]:
# Merging the 2024 teams information with the corresponding team statistics from team_dataframes[2024]
merged_df_2024 = seeds_df.merge(team_dataframes[2024], on='TeamID', how='left')

merged_df_2024

,Tournament,Seed,TeamID,WinRate,FGM,FGA,FGM3,FGA3,FTM,FTA,...,TO,Stl,Blk,PF,HighestRank,TopFourRank,BottomFourRank,MedianRank,LowestRank,FinalRankDay133
0,M,W01,1345,0.892857,29.142857,59.321429,8.464286,21.071429,18.071429,25.035714,...,11.000000,5.821429,3.857143,13.892857,27.0,1.0,3.0,2.0,1.0,1.0
1,M,W02,1266,0.785714,29.642857,61.285714,9.071429,25.535714,11.178571,15.535714,...,10.107143,9.000000,3.321429,15.357143,65.0,8.0,15.0,11.0,1.0,13.0
2,M,W03,1243,0.607143,24.750000,56.785714,7.428571,23.571429,15.642857,21.357143,...,14.178571,6.214286,4.250000,17.464286,152.0,51.0,71.0,61.0,18.0,73.0
3,M,W04,1397,0.785714,28.107143,61.357143,8.571429,24.535714,15.892857,21.285714,...,10.214286,8.000000,4.821429,17.500000,131.0,5.0,11.0,7.0,2.0,5.0
4,M,W05,1181,0.785714,28.464286,58.607143,8.500000,22.000000,15.107143,20.821429,...,9.678571,6.464286,3.607143,15.892857,121.0,11.0,24.0,15.0,3.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,W,Z12,3405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,W,Z13,3387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,W,Z14,3241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,W,Z15,3436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
M_merge = merged_df_2024[merged_df_2024['Tournament'].str.contains('M')]
W_merge = merged_df_2024[merged_df_2024['Tournament'].str.contains('W')]

In [61]:
M_merge

,Tournament,Seed,TeamID,WinRate,FGM,FGA,FGM3,FGA3,FTM,FTA,...,TO,Stl,Blk,PF,HighestRank,TopFourRank,BottomFourRank,MedianRank,LowestRank,FinalRankDay133
0,M,W01,1345,0.892857,29.142857,59.321429,8.464286,21.071429,18.071429,25.035714,...,11.000000,5.821429,3.857143,13.892857,27.0,1.0,3.0,2.0,1.0,1.0
1,M,W02,1266,0.785714,29.642857,61.285714,9.071429,25.535714,11.178571,15.535714,...,10.107143,9.000000,3.321429,15.357143,65.0,8.0,15.0,11.0,1.0,13.0
2,M,W03,1243,0.607143,24.750000,56.785714,7.428571,23.571429,15.642857,21.357143,...,14.178571,6.214286,4.250000,17.464286,152.0,51.0,71.0,61.0,18.0,73.0
3,M,W04,1397,0.785714,28.107143,61.357143,8.571429,24.535714,15.892857,21.285714,...,10.214286,8.000000,4.821429,17.500000,131.0,5.0,11.0,7.0,2.0,5.0
4,M,W05,1181,0.785714,28.464286,58.607143,8.500000,22.000000,15.107143,20.821429,...,9.678571,6.464286,3.607143,15.892857,121.0,11.0,24.0,15.0,3.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,M,Z12,1433,0.678571,24.142857,54.500000,8.464286,23.535714,15.500000,19.535714,...,11.607143,6.071429,4.392857,16.357143,204.0,78.0,100.0,90.0,38.0,77.0
60,M,Z13,1233,0.481481,26.333333,60.851852,8.074074,23.000000,12.148148,18.481481,...,9.629630,9.481481,3.037037,18.851852,293.0,168.0,212.0,187.0,67.0,188.0
61,M,Z14,1213,0.851852,26.111111,56.888889,6.962963,20.222222,19.333333,25.962963,...,12.222222,7.962963,4.888889,18.444444,141.0,31.0,67.0,52.0,4.0,59.0
62,M,Z15,1421,0.592593,25.629630,55.814815,7.629630,20.333333,18.185185,24.148148,...,12.000000,5.259259,4.185185,17.814815,327.0,145.0,191.0,161.0,88.0,147.0


In [62]:
import pandas as pd
from itertools import combinations

# 假设M_merge是已经定义并包含合并数据的DataFrame
# Create all unique pairs of seeds without repeating pairs (team against itself)
seed_pairs = list(combinations(M_merge['Seed'].unique(), 2))

# Initialize an empty DataFrame for the new matrix
columns = ['Team1_Seed', 'Team2_Seed'] + [f'Team1_{col}' for col in M_merge.columns if col not in ['Tournament', 'Seed', 'TeamID']] + [f'Team2_{col}' for col in M_merge.columns if col not in ['Tournament', 'Seed', 'TeamID']]
team_pro = pd.DataFrame(columns=columns)

# Populate the new matrix with the team stats for each pair
for seed1, seed2 in seed_pairs:
    team1_data = M_merge[M_merge['Seed'] == seed1].drop(columns=['Tournament', 'Seed', 'TeamID']).reset_index(drop=True)
    team2_data = M_merge[M_merge['Seed'] == seed2].drop(columns=['Tournament', 'Seed', 'TeamID']).reset_index(drop=True)
    
    # Join the two team's data
    combined_data = pd.concat([team1_data, team2_data], axis=1)
    
    # Add the seeds at the beginning
    combined_data.insert(0, 'Team1_Seed', seed1)
    combined_data.insert(1, 'Team2_Seed', seed2)
    
    # Rename columns to reflect team1 and team2
    combined_data.columns = columns
    
    # Append to the main DataFrame
    team_pro = pd.concat([team_pro, combined_data], ignore_index=True)

# Now 2024_team_pro will have the desired structure without teams playing against themselves
print(team_pro.head())


  Team1_Seed Team2_Seed  Team1_WinRate  Team1_FGM  Team1_FGA  Team1_FGM3  \
0        W01        W02       0.892857  29.142857  59.321429    8.464286   
1        W01        W03       0.892857  29.142857  59.321429    8.464286   
2        W01        W04       0.892857  29.142857  59.321429    8.464286   
3        W01        W05       0.892857  29.142857  59.321429    8.464286   
4        W01        W06       0.892857  29.142857  59.321429    8.464286   

   Team1_FGA3  Team1_FTM  Team1_FTA   Team1_OR  ...   Team2_TO  Team2_Stl  \
0   21.071429  18.071429  25.035714  11.464286  ...  10.107143   9.000000   
1   21.071429  18.071429  25.035714  11.464286  ...  14.178571   6.214286   
2   21.071429  18.071429  25.035714  11.464286  ...  10.214286   8.000000   
3   21.071429  18.071429  25.035714  11.464286  ...   9.678571   6.464286   
4   21.071429  18.071429  25.035714  11.464286  ...   9.928571   8.071429   

   Team2_Blk   Team2_PF  Team2_HighestRank  Team2_TopFourRank  \
0   3.321429  1

In [69]:
team_pro.columns

Index(['Team1_Seed', 'Team2_Seed', 'Team1_WinRate', 'Team1_FGM', 'Team1_FGA',
       'Team1_FGM3', 'Team1_FGA3', 'Team1_FTM', 'Team1_FTA', 'Team1_OR',
       'Team1_DR', 'Team1_Ast', 'Team1_TO', 'Team1_Stl', 'Team1_Blk',
       'Team1_PF', 'Team1_HighestRank', 'Team1_TopFourRank',
       'Team1_BottomFourRank', 'Team1_MedianRank', 'Team1_LowestRank',
       'Team1_FinalRankDay133', 'Team2_WinRate', 'Team2_FGM', 'Team2_FGA',
       'Team2_FGM3', 'Team2_FGA3', 'Team2_FTM', 'Team2_FTA', 'Team2_OR',
       'Team2_DR', 'Team2_Ast', 'Team2_TO', 'Team2_Stl', 'Team2_Blk',
       'Team2_PF', 'Team2_HighestRank', 'Team2_TopFourRank',
       'Team2_BottomFourRank', 'Team2_MedianRank', 'Team2_LowestRank',
       'Team2_FinalRankDay133'],
      dtype='object')

In [67]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Assuming 'team_dataframes' is a dictionary with DataFrames for each year from 2003 to 2023
# and 'team_pro' is the DataFrame for predictions

# Step 1: Data Preprocessing

# Concatenating all DataFrames into one
all_data = pd.concat(newnewdata_by_season.values(), ignore_index=True)

# Separating features and target
X = all_data.drop('result', axis=1)
y = all_data['result']


In [68]:
all_data.columns

Index(['team_1', 'team_2', 'team_1_WinRate', 'team_1_FGM', 'team_1_FGA',
       'team_1_FGM3', 'team_1_FGA3', 'team_1_FTM', 'team_1_FTA', 'team_1_OR',
       'team_1_DR', 'team_1_Ast', 'team_1_TO', 'team_1_Stl', 'team_1_Blk',
       'team_1_PF', 'team_1_HighestRank', 'team_1_TopFourRank',
       'team_1_BottomFourRank', 'team_1_MedianRank', 'team_1_LowestRank',
       'team_1_FinalRankDay133', 'team_2_WinRate', 'team_2_FGM', 'team_2_FGA',
       'team_2_FGM3', 'team_2_FGA3', 'team_2_FTM', 'team_2_FTA', 'team_2_OR',
       'team_2_DR', 'team_2_Ast', 'team_2_TO', 'team_2_Stl', 'team_2_Blk',
       'team_2_PF', 'team_2_HighestRank', 'team_2_TopFourRank',
       'team_2_BottomFourRank', 'team_2_MedianRank', 'team_2_LowestRank',
       'team_2_FinalRankDay133', 'result'],
      dtype='object')

In [66]:

# Data scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Preprocessing data for 2024 predictions, ensuring feature names and order match
team_pro_aligned = team_pro[X.columns.tolist()]  # Align feature names and order
X_pro_scaled = scaler.transform(team_pro_aligned)  # Scale prediction data

# Converting data to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float)
y_tensor = torch.tensor(y.values, dtype=torch.float).view(-1, 1)
X_pro_tensor = torch.tensor(X_pro_scaled, dtype=torch.float)

# Step 2: Defining the Model

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True, num_layers=2)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return torch.sigmoid(out)

# Model instance
model = RNNModel(input_dim=42, hidden_dim=64, output_dim=1)

# Step 3: Data Loading

dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Step 4: Model Training

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Step 5: Making Predictions

with torch.no_grad():
    predictions = model(X_pro_tensor)
    print(predictions.numpy())


KeyError: "None of [Index(['team_1', 'team_2', 'team_1_WinRate', 'team_1_FGM', 'team_1_FGA',\n       'team_1_FGM3', 'team_1_FGA3', 'team_1_FTM', 'team_1_FTA', 'team_1_OR',\n       'team_1_DR', 'team_1_Ast', 'team_1_TO', 'team_1_Stl', 'team_1_Blk',\n       'team_1_PF', 'team_1_HighestRank', 'team_1_TopFourRank',\n       'team_1_BottomFourRank', 'team_1_MedianRank', 'team_1_LowestRank',\n       'team_1_FinalRankDay133', 'team_2_WinRate', 'team_2_FGM', 'team_2_FGA',\n       'team_2_FGM3', 'team_2_FGA3', 'team_2_FTM', 'team_2_FTA', 'team_2_OR',\n       'team_2_DR', 'team_2_Ast', 'team_2_TO', 'team_2_Stl', 'team_2_Blk',\n       'team_2_PF', 'team_2_HighestRank', 'team_2_TopFourRank',\n       'team_2_BottomFourRank', 'team_2_MedianRank', 'team_2_LowestRank',\n       'team_2_FinalRankDay133'],\n      dtype='object')] are in the [columns]"

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# 假设已经有了DataFrame: team_dataframes['2003']到team_dataframes['2023']
# 假设 team_pro 是预测用的DataFrame

# 步骤1: 数据预处理

all_data = pd.concat(newnewdata_by_season.values(), ignore_index=True)
X = all_data.drop('result', axis=1)
y = all_data['result']
    # 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 为2024年数据做预处理
X_pro = team_pro

X_pro_scaled = scaler.transform(X_pro)  # Scaling the prediction data

# Correcting the tensor conversion
X_tensor = torch.tensor(X_scaled, dtype=torch.float)  # Convert scaled training features to tensor
y_tensor = torch.tensor(y.values, dtype=torch.float).view(-1, 1)  # Convert target to tensor
X_pro_tensor = torch.tensor(X_pro_scaled, dtype=torch.float)  # Convert scaled prediction features to tensor


# 步骤2: 定义模型（如前所述）


import torch
import torch.nn as nn
import torch.optim as optim

# 假设输入数据维度是42（因为最后一列是我们要预测的结果），输出是二分类问题（胜/负）
input_dim = 42
hidden_dim = 64  # 隐藏层维度，这可以根据需要调整
output_dim = 1   # 输出层维度，用于二分类问题

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True, num_layers=2)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x的维度: [batch_size, seq_length, input_dim]
        out, _ = self.rnn(x)
        # 取RNN的最后一次输出
        out = out[:, -1, :]
        out = self.fc(out)
        return torch.sigmoid(out)  # 使用sigmoid激活函数因为是二分类问题

# 实例化模型
model = RNNModel(input_dim, hidden_dim, output_dim)

# 打印模型结构
print(model)

# 步骤3: 数据加载
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# 步骤4: 模型训练
model = RNNModel(input_dim=42, hidden_dim=64, output_dim=1)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 步骤5: 预测
with torch.no_grad():
    predictions = model(X_pro_tensor)
    # 输出为胜率的概率
    print(predictions.numpy())



ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Team1_Ast
- Team1_Blk
- Team1_BottomFourRank
- Team1_DR
- Team1_FGA
- ...
Feature names seen at fit time, yet now missing:
- team_1
- team_1_Ast
- team_1_Blk
- team_1_BottomFourRank
- team_1_DR
- ...
